In [ ]:
import pandas as pd
from collections import Counter
from itertools import combinations

In [ ]:
data = pd.read_csv('movie_bd_v5.csv') 

In [ ]:
data.info()

In [ ]:
answers = {} # создадим словарь для ответов

# Предобработка

In [ ]:
# на этапе рефакторинга для удобства было принято решение предварительно совершить несколько манипуляций
data['profit'] = data.revenue - data.budget # под «прибылью» или «убытками» понимается разность между сборами и бюджетом

# представляем данные в виде списка
data['genres'] = data.genres.str.split('|') # жанры 
data['cast'] = data.cast.str.split('|') # актеров
data['director'] = data.director.str.split('|') # режиссеров 

data['release_date'] = data.release_date.str.split('/') # дата релиза [month, day, year]
data['release_month'] = data.release_date.apply(lambda x: x[0]) # номер месяца релиза выношу отдельно

# 1. У какого фильма из списка самый большой бюджет?

In [ ]:
# так записываем свой вариант ответа
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [ ]:
# тут пишем код для решения данного вопроса:
data.original_title[data.budget == data.budget.max()]

In [ ]:
# можно добавлять разные варианты решения
data['original_title'][(data['budget'] == data['budget'].max())]

# 2. Какой из фильмов самый длительный (в минутах)?

In [ ]:
answers['2'] = '2. Gods and Generals (tt0279111)'

In [ ]:
data.original_title[data.runtime == data.runtime.max()]

# 3. Какой из фильмов самый короткий (в минутах)?





In [ ]:
answers['3'] = '3. Winnie the Pooh (tt1449283)'
data.original_title[data.runtime == data.runtime.min()]

# 4. Какова средняя длительность фильмов?


In [ ]:
answers['4'] = '2. 110'
round(data.runtime.mean())

# 5. Каково медианное значение длительности фильмов? 

In [ ]:
answers['5'] = '1. 107'
round(data.runtime.median())

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [ ]:
answers['6'] = '5. Avatar (tt0499549)'
data.original_title[data.profit == data.profit.max()]

# 7. Какой фильм самый убыточный? 

In [ ]:
answers['7'] = '5. The Lone Ranger (tt1210819)'
data.original_title[data.profit == data.profit.min()]

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [ ]:
answers['8'] = '1. 1478'
data.original_title[data.profit > 0].count()

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [ ]:
answers['9'] = '4. The Dark Knight (tt0468569)'
df = data[data.release_year == 2008] # создаю новую ссылку на датафрейм где релиз фильмов только 2008 год
df.original_title[df.revenue == df.revenue.max()]

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [ ]:
answers['10'] = '5. The Lone Ranger (tt1210819)'
df = data[(data.release_year >= 2012) & (data.release_year <= 2014)]
df.original_title[df.profit == df.profit.min()]

# 11. Какого жанра фильмов больше всего?

In [ ]:
# эту задачу тоже можно решать разными подходами
answers['11'] = '3. Drama'
Counter(data.genres.sum()).most_common(1) # объект Counter получает список всех жанров всех  фильмов

ВАРИАНТ 2

In [ ]:
pd.Series(data.genres.sum()).value_counts().head(1) # создаем новый объект Series из списка всех жанров всех фильмов и считаем сколько раз встречается каждый жанр

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [ ]:
answers['12'] = '1. Drama'
pd.Series(data.genres[data.profit > 0].sum()).value_counts().head(1)

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [ ]:
answers['13'] = '5. Peter Jackson'
df = data.explode('director').copy() # датафрейм где каждому режиссеру соответствует фильм
df.groupby(['director'])['revenue'].sum().sort_values(ascending=False).head(1) # группируем этот датафрейм по режиссерам и считаем суммарные сборы

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [ ]:
answers['14'] = '3. Robert Rodriguez'
# использую метод contains для определения жанра Action в фильме
df = data[data.genres.apply(lambda x: '|'.join(x)).str.contains('Action', na=False)].copy() # датафрейм фильмов жанр которых включаем в себя Action
df.director.explode().value_counts().head(1) # считаем каждого режиссера который принимал участие в создании Action фильма

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [ ]:
answers['15'] = '3. Chris Hemsworth'
df = data[data.release_year == 2012].copy() # датафрейм фильмов 2012 года
# датафрейм где каждому актеру соответствует фильм группируем по актерам и считаем суммарные сборы
df.explode('cast').groupby(['cast']).revenue.sum().sort_values(ascending=False).head(1)

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [ ]:
answers['16'] = '3. Matt Damon'
df = data[data.budget > data.budget.mean()].copy() # датафрейм высокобюджетных фильмов
df.explode('cast').cast.value_counts().head(1)

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [ ]:
answers['17'] = '2. Action'
df = data.explode('cast').copy() # датафрейм где каждому актеру соответствует фильм
# объект Counter получает список жанров фильмов в которых снимался Nicolas Cage
Counter(df.groupby(['cast']).genres.sum()['Nicolas Cage']).most_common(1)

# 18. Самый убыточный фильм от Paramount Pictures

In [ ]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626)'
df = data[data.production_companies.str.contains('Paramount Pictures')].copy() # датафрейм где фильмы в создании которых принимала участие компания Paramount Pictures
df[df.profit == df.profit.min()].original_title

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [ ]:
answers['19'] = '5. 2015'
data.groupby(['release_year']).revenue.sum().sort_values(ascending=False).head(1) # группируем по году релиза и считаем суммарный профит

# 20. Какой самый прибыльный год для студии Warner Bros?

In [ ]:
answers['20'] = '1. 2014'
df = data[data.production_companies.str.contains('Warner Bros')].copy() # датафрейм который состоит из фильмов в создании которых принимала участие компания Warner Bros
df.groupby(['release_year']).profit.sum().sort_values(ascending=False).head(1) 

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [ ]:
answers['21'] = '4. Сентябрь'
data.release_month.value_counts().head(1)

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [ ]:
answers['22'] = '2. 450'
data.release_month.value_counts()['6'] + data.release_month.value_counts()['7'] + data.release_month.value_counts()['8']

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [ ]:
answers['23'] = '5. Peter Jackson'
df = data.explode('director').copy() # новый датафрейм где отдельному режиссеру соответствует фильм
df['release_month'] = pd.to_numeric(data['release_month']) # object в int
df[(df.release_month == 12) | (df.release_month == 1) | (df.release_month == 2)].director.value_counts().head(1)

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [ ]:
answers['24'] = '5. Four By Two Productions'
df['title_symbols'] = df.original_title.apply(lambda x: len(x)) # столбец количества символов в названии
df['production_companies'] = df.production_companies.str.split('|') # представляем данные о компаниях которые принимали участие в создании фильма в виде списка
ndf = df.explode('production_companies').copy() # новый датафрейм в котором отображено участие каждой компании в производстве фильма
ndf.groupby(['production_companies'])['title_symbols'].mean().sort_values(ascending=False).head(1) # группируем по названию компании, вычисляем среднее количество символов и сортируем

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [ ]:
answers['25'] = '3. Midnight Picture Show'
df['overview'] = df.overview.str.split(' ') # описание фильма представляю как список слов
df['overview_words'] = df.overview.apply(lambda x: len(x)) # для каждого списка слов ставлю в соответствие его длинну 
ndf = df.explode('production_companies').copy() 
ndf.groupby(['production_companies'])['overview_words'].mean().sort_values(ascending=False).head(1) 

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [ ]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave'
# список фильмов, которые имеют рейтинг больше чем 99% фильмов
data[data.vote_average > data.vote_average.quantile(q=0.99)].original_title.tolist()

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [ ]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint'
# с помощью модуля itertools и функции combinations получаю всевозможные комбинации из 2х актеров в каждом фильме
Counter(data.cast.apply(lambda x: list(combinations(x, 2))).sum()).most_common(1)

# Submission

In [ ]:
# в конце можно посмотреть мои ответы к каждому вопросу
answers

In [ ]:
# и убедиться что ни чего не пропустил)
len(answers)